In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="ticks", color_codes=True)
import math

import sklearn.linear_model as slmodel
from sklearn import datasets

# 1. Примеры дамми-переменных

1. Можно привести массу примеров подобных переменных при оценивании стоимости различной продукций, например, б/у - не б/у (для любых товаров), последняя модель - модель старшего поколения(в основном для электроники, автомобилей), импортное производство - отечественное и т.д.

2. При моделировании вероятности заражения/заболевания какой-нибудь болезнью можно использовать параметры: делал прививку/не делал или проходил медобследование за последний год/не проходил.

3. Прогноз успеваемости студента в сессию: посещал занятия/не посещал, сдавал дз/не сдавал, имел академический долг за последний семестр/не имел и т.п.

# 2. Гипотеза о незначимости регрессии

Вернемся к таблице со статистикой заболеваемости Covid-19

In [2]:
data = pd.read_csv('data.csv')
data = data[['Страна', 'Всего случаев', 'Всего смертей', 'Сделано тестов', 'Население']][2:].reset_index(drop=True)

data['Всего случаев'] = data['Всего случаев'].apply(lambda x: str(x).replace(' ', '')).astype(int)
data['Всего смертей'] = data['Всего смертей'].apply(lambda x: str(x).replace(' ', '')).astype(int)
data['Сделано тестов'] = data['Сделано тестов'].apply(lambda x: str(x).replace(' ', '')).astype(int)
data['Население'] = data['Население'].apply(lambda x: str(x).replace(' ', '')).astype(int)

In [4]:
data.head()

,Страна,Всего случаев,Всего смертей,Сделано тестов,Население
0,Россия,362342,3807,9160590,145928655
1,Испания,283339,27117,3556567,46753098
2,Великобритания,265227,37048,3681295,67852019
3,Италия,230555,32955,3539927,60469988
4,Франция,182722,28530,1384633,65259974


На этот раз рассмотрим модель зависимость смертей от кол-ва случаев, кол-ва проведенных тестов и численности населения страны

In [13]:
y = data['Всего смертей'].values
x = data[['Всего случаев', 'Население', 'Сделано тестов']].values

In [7]:
model = slmodel.LinearRegression()
model.fit(x, y)
model.intercept_, model.coef_

(-103.91472036813911,
 array([ 1.34047106e-01, -4.28807527e-06, -3.57657870e-03]))

In [25]:
# Параметры данной модели:
n = len(data)
k = 4
# уровень значимости альфа
alpha = 0.05

Незначимость модели можно оценить с помощью F-теста: 
$$ F_{model} = \dfrac{ESS/(k-1)}{RSS/(n-k)} \sim F_{k-1,n-k} $$

где $ ESS = TSS - RSS $

Нулевая гипотеза: $$ H_0: \beta_1 = \beta_2 = \beta_3 = 0 $$

отвергаеется, если $ F_{model} > F_{k-1,n-k} $

In [23]:
TSS = np.sum((y - data['Всего смертей'].mean())**2)

y_predict = model.intercept_ + x.dot(model.coef_)
RSS = np.sum((y - y_predict)**2)

In [26]:
from scipy.stats import f

F_model = ((TSS - RSS)/(k - 1))/(RSS/(n - k))
F = f.ppf(1 - alpha, k-1, n-k)

In [27]:
print(F_model, F)

98.70261333201977 2.6771069890604258


Получили, что $ F_{model} > F_{k-1,n-k} $, т.е. гипотеза о незначимости регресси отвергается.